## Image Classifier 

In [0]:
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

#### Loading and preparing dataset

In [0]:
dataset , info = tfds.load('mnist',with_info=True, as_supervised=True)

####Training and Test Sets

In [0]:
mnist_train, mnist_test = dataset['train'], dataset['test']

#### Converting samples from integers to floating-point numbers.

In [0]:
def convert_types(image, label):
  image = tf.cast(image, tf.float32)
  image /= 255
  return image, label

In [0]:
mnist_train = mnist_train.map(convert_types).shuffle(10000).batch(32)
mnist_test = mnist_test.map(convert_types).batch(32)

#### Building model using KERAS model Subclass API

In [0]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10, activation='softmax')

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)
  
model = MyModel()

#### Optimizer and Loss Function

In [0]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [0]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name = 'test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

#### Train and Test Functions

In [0]:
## Function for Testing the model
def test_step(image, label):
  predictions = model(image)
  t_loss = loss_object(label, predictions)
  
  test_loss(t_loss)
  test_accuracy(label, predictions)

In [0]:
## Function for training model
def train_step(image, label):
  with tf.GradientTape() as tape:
    predictions = model(image)
    loss = loss_object(label, predictions)
  gradients = tape.gradient(loss,model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  
  train_loss(loss)
  train_accuracy(label, predictions)

### NOW TRAINING AND TESTING

In [42]:
EPOCHS = 5

for epoch in range(EPOCHS):
  for image, label in mnist_train:
    train_step(image, label)
    
  for test_image, test_label in mnist_test:
    test_step(test_image, test_label)
    
  template = 'EPOCH: {}, LOSS: {}, ACCURACY: {}, Test Loss: {}, Test Accuracy: {}'
  print(template.format(epoch+1, train_loss.result(),
                       train_accuracy.result()*100,
                       test_loss.result(),
                       test_accuracy.result()*100))

EPOCH: 1, LOSS: 0.03155630826950073, ACCURACY: 99.02726745605469, Test Loss: 0.06482817232608795, Test Accuracy: 98.24874877929688
EPOCH: 2, LOSS: 0.028559476137161255, ACCURACY: 99.11962127685547, Test Loss: 0.06584858894348145, Test Accuracy: 98.27444458007812
EPOCH: 3, LOSS: 0.026086557656526566, ACCURACY: 99.19664001464844, Test Loss: 0.06747660040855408, Test Accuracy: 98.2770004272461
EPOCH: 4, LOSS: 0.024069175124168396, ACCURACY: 99.26005554199219, Test Loss: 0.07151933759450912, Test Accuracy: 98.24909210205078
EPOCH: 5, LOSS: 0.02238628827035427, ACCURACY: 99.31170654296875, Test Loss: 0.07356638461351395, Test Accuracy: 98.25250244140625


## ACCURACY 99.3%